In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='3'
import tensorflow as tf
import random
import time
import datetime
# from test import test
from dataloader_yun import DataManager
# from generator import Generator
# from discriminator import Discriminator
# from rollout import ROLLOUT
import numpy as np
# from discriminator import Discriminator
import re

import gensim

In [2]:
# data_dir = '/home/yunzhao/PJ/seqGan/AC_SEQ_GAN/PCNN_GAN/Discriminator/data'
data_dir = '/home/rohanjain/data/sentgen/'
word2vec_file = os.path.join(data_dir, 'vec.txt')
relation2id_file = os.path.join(data_dir, 'RE/relation2id.txt')
datamanager = DataManager(100, word2vec_file, relation2id_file)
entity2id_file = os.path.join(data_dir, 'RE/entity2id.txt')
training_data = datamanager.load_training_data(entity2id_file,filename=os.path.join(data_dir, 'RE/train.txt'))
training_data = np.array(training_data)
testing_data = datamanager.load_testing_data(filename=os.path.join(data_dir, 'RE/test.txt'))
print(str(len(training_data))+" "+str(len(testing_data)))


RelationTotal: 53
WordTotal:  114043
Word dimension:  50
Start loading training data.
Start loading testing data.
570088 96678


In [3]:
# model = gensim.models.Word2Vec.load_word2vec_format(word2vec_file)
model = gensim.models.KeyedVectors.load_word2vec_format(word2vec_file)

In [15]:
all_word2vec_keys = list(model.vocab.keys())

In [5]:
sample=training_data[10]
sample

Sentence(entity1=jorge_amado| entity2=brazil| relation=Relation(id=28| name=/people/person/nationality| number=0)| words=mr. rabassa is a renowned translator , of gabriel garcía márquez , jorge_amado and mario vargas llosa -- and of lispector , who became , in the mid-20th century , one of brazil 's most influential writers , described as the kafka of latin american fiction .)

In [6]:
sample.words

['mr.',
 'rabassa',
 'is',
 'a',
 'renowned',
 'translator',
 ',',
 'of',
 'gabriel',
 'garcía',
 'márquez',
 ',',
 'jorge_amado',
 'and',
 'mario',
 'vargas',
 'llosa',
 '--',
 'and',
 'of',
 'lispector',
 ',',
 'who',
 'became',
 ',',
 'in',
 'the',
 'mid-20th',
 'century',
 ',',
 'one',
 'of',
 'brazil',
 "'s",
 'most',
 'influential',
 'writers',
 ',',
 'described',
 'as',
 'the',
 'kafka',
 'of',
 'latin',
 'american',
 'fiction',
 '.']

def clean_str_old(string):
    """
    Tokenization/string cleaning for all datasets except for SST.
    Original taken from https://github.com/yoonkim/CNN_sentence/blob/master/process_data.py
    """
    #string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)
    string = re.sub(r"[^A-Za-z_]", " ", string)
    string = re.sub(r"\'s", " \'s", string)
    string = re.sub(r"\'ve", " \'ve", string)
    string = re.sub(r"n\'t", " n\'t", string)
    string = re.sub(r"\'re", " \'re", string)
    string = re.sub(r"\'d", " \'d", string)
    string = re.sub(r"\'ll", " \'ll", string)
    string = re.sub(r",", " , ", string)
    string = re.sub(r"!", " ! ", string)
    string = re.sub(r"\(", " \( ", string)
    string = re.sub(r"\)", " \) ", string)
    string = re.sub(r"\?", " \? ", string)
    string = re.sub(r"\s{2,}", " ", string)
    string = re.sub(r",", "", string)
    return string.strip().lower()

def clean_str(string):
    """
    Tokenization/string cleaning for all datasets except for SST.
    Original taken from https://github.com/yoonkim/CNN_sentence/blob/master/process_data.py
    """
    #string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)
    string = re.sub(r"[^A-Za-z_]", " ", string)
    string = re.sub(r"\'s", " \'s", string)
    string = re.sub(r"\'ve", " \'ve", string)
    string = re.sub(r"n\'t", " n\'t", string)
    string = re.sub(r"\'re", " \'re", string)
    string = re.sub(r"\'d", " \'d", string)
    string = re.sub(r"\'ll", " \'ll", string)
    string = re.sub(r",", " , ", string)
    string = re.sub(r"!", " ! ", string)
    string = re.sub(r"\(", " \( ", string)
    string = re.sub(r"\)", " \) ", string)
    string = re.sub(r"\?", " \? ", string)
    string = re.sub(r"\s{2,}", " ", string)
    string = re.sub(r",", "", string)
    return string.strip().lower()

for sample in training_data:
    sample.words=clean_str(' '.join(sample.words))

In [7]:
sample.relation.id

28

In [8]:
sample.entity1

'jorge_amado'

In [9]:
sample.entity2

'brazil'

In [10]:
def get_entity_loc(d):
    e1 = d.entity1
    e2 = d.entity2
    # words = d.words.split()
    words = d.words
    l1 = 0
    l2 = 0
    for i, w in enumerate(words):
        if w == e1:
            l1 = i
        if w == e2:
            l2 = i
    return (l1,l2)

In [11]:
get_entity_loc(sample)

(12, 32)

In [12]:
all_words = []
type(all_words)
for sample in training_data:
    # all_words.extend(sample.words.split())
    all_words.extend(sample.words)

In [13]:
len(all_words)

22853774

In [14]:
len(set(all_words))

220460

# Set difference between all_words and all_word2vec_keys

In [17]:
len(list(set(all_words) - set(all_word2vec_keys)))

131252

# Now create histogram of vocab words

In [18]:
from collections import Counter
counter1 = Counter(all_words)

In [19]:
top_words = counter1.most_common(6000)

In [20]:
top_words[:50]

[(',', 1937054),
 ('the', 1098869),
 ('and', 710004),
 ('of', 638342),
 ('in', 581733),
 ('.', 564125),
 ('a', 438657),
 ('to', 408172),
 ("''", 306322),
 ("'s", 228455),
 ('for', 157847),
 ('that', 155391),
 ('on', 149467),
 (';', 142714),
 ('-lrb-', 142085),
 ('-rrb-', 142059),
 ('by', 135836),
 ('with', 135633),
 ('at', 134670),
 ('is', 122553),
 ('from', 116082),
 ('as', 107420),
 ('his', 105916),
 ('was', 101924),
 ('he', 85874),
 ('who', 85357),
 ('--', 80661),
 ('said', 77374),
 ('an', 74845),
 ('has', 65861),
 ('mr.', 64905),
 ('it', 62486),
 ('new', 61162),
 (':', 58811),
 ('have', 48523),
 ('are', 47180),
 ('be', 45279),
 ('had', 44485),
 ('but', 43705),
 ('its', 42900),
 ('which', 42157),
 ('not', 39558),
 ('this', 37759),
 ('her', 37473),
 ('about', 37453),
 ('will', 35639),
 ('president', 33986),
 ('one', 33673),
 ('like', 33598),
 ('their', 33578)]

# Now remove the top 5000 words in top_words that are not present in all_word2vec_keys

In [26]:
filt_top_words = []
for item in top_words.__iter__():
    # print(item)
    if item[0] in all_word2vec_keys:
        filt_top_words.append(item[0])

In [27]:
filt_top_words

[',',
 'the',
 'and',
 'of',
 'in',
 '.',
 'a',
 'to',
 'for',
 'that',
 'on',
 ';',
 'by',
 'with',
 'at',
 'is',
 'from',
 'as',
 'his',
 'was',
 'he',
 'who',
 'said',
 'an',
 'has',
 'it',
 'new',
 ':',
 'have',
 'are',
 'be',
 'had',
 'but',
 'its',
 'which',
 'not',
 'this',
 'her',
 'about',
 'will',
 'president',
 'one',
 'like',
 'their',
 'american',
 'after',
 'two',
 'or',
 'were',
 'new_york',
 'china',
 'when',
 'last',
 'would',
 'they',
 'more',
 'been',
 '$',
 'iraq',
 'years',
 'other',
 'united',
 'first',
 'york',
 'also',
 'states',
 'up',
 'she',
 'where',
 'brooklyn',
 'than',
 'former',
 'bush',
 'over',
 'city',
 'year',
 'into',
 'all',
 'out',
 'i',
 "'",
 'united_states',
 'most',
 'state',
 'there',
 'three',
 'including',
 'john',
 'wife',
 'manhattan',
 'some',
 'world',
 'now',
 'company',
 'washington',
 'between',
 'house',
 'time',
 'against',
 'through',
 'people',
 'before',
 'we',
 'david',
 'if',
 'him',
 'war',
 'many',
 'home',
 'what',
 'only',

In [32]:
vocab = filt_top_words[:5000]

In [33]:
vocab

[',',
 'the',
 'and',
 'of',
 'in',
 '.',
 'a',
 'to',
 'for',
 'that',
 'on',
 ';',
 'by',
 'with',
 'at',
 'is',
 'from',
 'as',
 'his',
 'was',
 'he',
 'who',
 'said',
 'an',
 'has',
 'it',
 'new',
 ':',
 'have',
 'are',
 'be',
 'had',
 'but',
 'its',
 'which',
 'not',
 'this',
 'her',
 'about',
 'will',
 'president',
 'one',
 'like',
 'their',
 'american',
 'after',
 'two',
 'or',
 'were',
 'new_york',
 'china',
 'when',
 'last',
 'would',
 'they',
 'more',
 'been',
 '$',
 'iraq',
 'years',
 'other',
 'united',
 'first',
 'york',
 'also',
 'states',
 'up',
 'she',
 'where',
 'brooklyn',
 'than',
 'former',
 'bush',
 'over',
 'city',
 'year',
 'into',
 'all',
 'out',
 'i',
 "'",
 'united_states',
 'most',
 'state',
 'there',
 'three',
 'including',
 'john',
 'wife',
 'manhattan',
 'some',
 'world',
 'now',
 'company',
 'washington',
 'between',
 'house',
 'time',
 'against',
 'through',
 'people',
 'before',
 'we',
 'david',
 'if',
 'him',
 'war',
 'many',
 'home',
 'what',
 'only',

# Write to file

In [36]:
file=open('vocab2id.txt','w')
i=0
file.write('START'+'\t'+str(i)+'\n')
i+=1
file.write('UNK'+'\t'+str(i)+'\n')
i+=1
file.write('PAD'+'\t'+str(i)+'\n')
for word in vocab:
    i+=1
    file.write(word+'\t'+str(i)+'\n')
file.close()